# Phase 8: Defensive Leverage Backtest

Test the volatility-aware RL agent on Nov-Dec 2025 downtrend.
Expected: Agent parks USDT during high-vol, selectively levers 3-5x on bounces.

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from data_fetcher import DataFetcher
from portfolio import Portfolio
from orchestrator import RLOrchestrator
from risk_manager import RiskManager
from strategies.dip_buy_lstm import generate_dip_signals, detect_dip_signal

print("Phase 8: Defensive Leverage Backtest")
print("="*50)

## 1. Fetch Nov-Dec 2025 Data

In [ ]:
fetcher = DataFetcher()

# Fetch extended data for volatility analysis
data = {}
for sym in ['XRP/USDT', 'BTC/USDT']:
    print(f"Fetching {sym}...")
    df = fetcher.fetch_ohlcv('kraken', sym, '1h', 2000)
    if not df.empty:
        data[sym] = df
        print(f"  {len(df)} candles from {df.index[0]} to {df.index[-1]}")

# Show price range
if 'XRP/USDT' in data:
    xrp = data['XRP/USDT']
    print(f"\nXRP range: ${xrp['close'].min():.4f} - ${xrp['close'].max():.4f}")
    print(f"XRP current: ${xrp['close'].iloc[-1]:.4f}")

## 2. Volatility Analysis

In [ ]:
risk = RiskManager(max_drawdown=0.20, max_leverage=10.0)

# Calculate rolling volatility
if 'XRP/USDT' in data:
    df = data['XRP/USDT']
    
    vol_history = []
    for i in range(20, len(df)):
        high = df['high'].iloc[:i+1].values
        low = df['low'].iloc[:i+1].values
        close = df['close'].iloc[:i+1].values
        atr_pct = risk.calculate_atr_pct(high, low, close)
        vol_history.append({'timestamp': df.index[i], 'atr_pct': atr_pct})
    
    vol_df = pd.DataFrame(vol_history)
    vol_df.set_index('timestamp', inplace=True)
    
    # Plot volatility
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    axes[0].plot(df.index, df['close'], label='XRP Price')
    axes[0].set_title('XRP/USDT Price')
    axes[0].set_ylabel('Price ($)')
    axes[0].legend()
    
    axes[1].plot(vol_df.index, vol_df['atr_pct'] * 100, label='ATR %', color='orange')
    axes[1].axhline(y=5, color='red', linestyle='--', label='High Vol Threshold')
    axes[1].axhline(y=8, color='darkred', linestyle='--', label='Extreme Vol')
    axes[1].set_title('Volatility (ATR %)')
    axes[1].set_ylabel('ATR %')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Stats
    print(f"\nVolatility Stats:")
    print(f"  Mean ATR%: {vol_df['atr_pct'].mean()*100:.2f}%")
    print(f"  Max ATR%: {vol_df['atr_pct'].max()*100:.2f}%")
    print(f"  % time in high vol: {(vol_df['atr_pct'] > 0.05).mean()*100:.1f}%")
    print(f"  % time in extreme vol: {(vol_df['atr_pct'] > 0.08).mean()*100:.1f}%")

## 3. Dip Detection Analysis

In [ ]:
# Find all dips in the period
if 'XRP/USDT' in data:
    df = data['XRP/USDT']
    
    dips = []
    for i in range(50, len(df)):
        close = df['close'].iloc[:i+1].values
        volume = df['volume'].iloc[:i+1].values
        high = df['high'].iloc[:i+1].values
        low = df['low'].iloc[:i+1].values
        
        signal = detect_dip_signal(close, volume, high, low)
        
        if signal['is_dip']:
            dips.append({
                'timestamp': df.index[i],
                'price': df['close'].iloc[i],
                'confidence': signal['confidence'],
                'rsi': signal['rsi'],
                'leverage_ok': signal['leverage_ok'],
                'atr_pct': signal['atr_pct']
            })
    
    dips_df = pd.DataFrame(dips)
    print(f"\nFound {len(dips)} dip signals")
    
    if len(dips) > 0:
        print(f"\nHigh-confidence dips (>0.6):")
        high_conf = dips_df[dips_df['confidence'] > 0.6]
        print(high_conf.to_string())
        
        print(f"\nLeverage-approved dips:")
        lev_ok = dips_df[dips_df['leverage_ok']]
        print(f"  Count: {len(lev_ok)}")
        if len(lev_ok) > 0:
            print(lev_ok.to_string())

## 4. RL Orchestrator Backtest

In [ ]:
# Initialize portfolio and orchestrator
starting = {'USDT': 1000.0, 'XRP': 500.0, 'BTC': 0.0}
portfolio = Portfolio(starting.copy())

try:
    orchestrator = RLOrchestrator(portfolio, data)
    rl_enabled = orchestrator.enabled
    print(f"\nRL Orchestrator: {'Enabled' if rl_enabled else 'Disabled'}")
    if rl_enabled:
        print(f"  Targets: {orchestrator.get_target_allocation()}")
except Exception as e:
    print(f"Could not initialize orchestrator: {e}")
    rl_enabled = False

In [ ]:
if rl_enabled and 'XRP/USDT' in data:
    df = data['XRP/USDT']
    btc_df = data.get('BTC/USDT')
    
    # Backtest loop
    results = []
    actions_log = []
    
    step_size = 4  # Every 4 hours
    
    for i in range(100, len(df), step_size):
        timestamp = df.index[i]
        xrp_price = df['close'].iloc[i]
        btc_price = btc_df['close'].iloc[i] if btc_df is not None and i < len(btc_df) else 90000.0
        
        prices = {'XRP': xrp_price, 'BTC': btc_price, 'USDT': 1.0}
        
        # Update env step
        if orchestrator.env is not None:
            orchestrator.env.current_step = min(i, orchestrator.env.max_steps - 1)
        
        # Get RL decision
        result = orchestrator.decide_and_execute(prices)
        orchestrator.check_and_manage_positions(prices)
        orchestrator.update_env_step()
        
        # Log
        total_value = portfolio.get_total_usd(prices)
        results.append({
            'timestamp': timestamp,
            'xrp_price': xrp_price,
            'portfolio_value': total_value,
            'usdt': portfolio.balances.get('USDT', 0),
            'volatility': result.get('volatility', 0),
            'market_state': result.get('market_state', 'unknown')
        })
        
        if result.get('executed'):
            actions_log.append({
                'timestamp': timestamp,
                'action': f"{result['asset']} {result['action_type']}",
                'leverage': result.get('leverage', 0),
                'price': xrp_price
            })
    
    results_df = pd.DataFrame(results)
    results_df.set_index('timestamp', inplace=True)
    
    print(f"\nBacktest complete: {len(results)} data points")
    print(f"Actions executed: {len(actions_log)}")

In [ ]:
if 'results_df' in dir() and len(results_df) > 0:
    # Plot results
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    
    # Portfolio value
    axes[0].plot(results_df.index, results_df['portfolio_value'], label='Portfolio Value')
    axes[0].set_title('Phase 8: Portfolio Value Over Time')
    axes[0].set_ylabel('Value ($)')
    axes[0].legend()
    
    # USDT balance
    axes[1].plot(results_df.index, results_df['usdt'], label='USDT Balance', color='green')
    axes[1].set_title('USDT Balance (Defensive Parking)')
    axes[1].set_ylabel('USDT')
    axes[1].legend()
    
    # Volatility
    axes[2].plot(results_df.index, results_df['volatility'] * 100, label='ATR %', color='orange')
    axes[2].axhline(y=5, color='red', linestyle='--', label='High Vol Threshold')
    axes[2].set_title('Market Volatility')
    axes[2].set_ylabel('ATR %')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
    # Summary stats
    initial_value = results_df['portfolio_value'].iloc[0]
    final_value = results_df['portfolio_value'].iloc[-1]
    pnl = final_value - initial_value
    roi = (pnl / initial_value) * 100
    
    print(f"\n" + "="*50)
    print(f"PHASE 8 DEFENSIVE BACKTEST RESULTS")
    print(f"="*50)
    print(f"Initial Value: ${initial_value:.2f}")
    print(f"Final Value:   ${final_value:.2f}")
    print(f"Total P&L:     ${pnl:+.2f} ({roi:+.1f}%)")
    print(f"")
    print(f"Actions Executed: {len(actions_log)}")
    print(f"Avg Volatility:   {results_df['volatility'].mean()*100:.2f}%")
    print(f"Time in Fear:     {(results_df['volatility'] > 0.05).mean()*100:.1f}%")

## 5. Action Analysis

In [ ]:
if 'actions_log' in dir() and len(actions_log) > 0:
    actions_df = pd.DataFrame(actions_log)
    
    print("\nAction Summary:")
    print(actions_df['action'].value_counts())
    
    print("\nLeveraged Trades:")
    lev_trades = actions_df[actions_df['leverage'] > 0]
    print(f"  Count: {len(lev_trades)}")
    if len(lev_trades) > 0:
        print(f"  Avg leverage: {lev_trades['leverage'].mean():.1f}x")
        print(lev_trades.to_string())
else:
    print("\nNo actions executed - agent stayed defensive")

## 6. Benchmark Comparison

In [ ]:
if 'XRP/USDT' in data and 'results_df' in dir():
    df = data['XRP/USDT']
    
    # Buy and hold benchmark
    initial_xrp = df['close'].iloc[100]
    final_xrp = df['close'].iloc[-1]
    xrp_return = (final_xrp - initial_xrp) / initial_xrp * 100
    
    # Our strategy return
    strategy_return = roi if 'roi' in dir() else 0
    
    print(f"\n" + "="*50)
    print(f"BENCHMARK COMPARISON")
    print(f"="*50)
    print(f"XRP Buy & Hold: {xrp_return:+.1f}%")
    print(f"Phase 8 Strategy: {strategy_return:+.1f}%")
    print(f"")
    print(f"Outperformance: {strategy_return - xrp_return:+.1f}%")
    
    if strategy_return > xrp_return:
        print(f"\n✓ Strategy outperformed buy & hold")
    else:
        print(f"\n✗ Strategy underperformed (needs more training)")